## Setup

In [1]:
import sys
from pathlib import Path

REPO_DIR = Path("/scratch/ucgd/lustre-labs/quinlan/u6018199/tapestry")
sys.path.append(str(REPO_DIR / "src" / "util"))
sys.path

['/usr/lib64/python311.zip',
 '/usr/lib64/python3.11',
 '/usr/lib64/python3.11/lib-dynload',
 '',
 '/scratch/ucgd/lustre-labs/quinlan/u6018199/tapestry/.venv/lib64/python3.11/site-packages',
 '/scratch/ucgd/lustre-labs/quinlan/u6018199/tapestry/.venv/lib64/python3.11/site-packages/cyvcf2-0.31.1-py3.11-linux-x86_64.egg',
 '/scratch/ucgd/lustre-labs/quinlan/u6018199/tapestry/.venv/lib/python3.11/site-packages',
 '/scratch/ucgd/lustre-labs/quinlan/u6018199/tapestry/.venv/lib/python3.11/site-packages/cyvcf2-0.31.1-py3.11-linux-x86_64.egg',
 '/scratch/ucgd/lustre-labs/quinlan/u6018199/tapestry/src/util']

## Palladium Trio

In [2]:
FATHER_UID = "NA12877" 
MOTHER_UID = "NA12878"
CHILD_UID = "NA12881" # 2211

## Founder-phased DNA methylation in Father, Mother, Child

In [3]:
PB_CPG_TOOL_MODE = 'model' # mode of aligned_bam_to_cpg_scores
# output dir of src/phase_meth_to_founder_haps.py
OUTPUT_DIR = Path(f"/scratch/ucgd/lustre-labs/quinlan/data-shared/dna-methylation/CEPH1463.GRCh38.hifi.{PB_CPG_TOOL_MODE}.founder-phased")
OUTPUT_DIR

PosixPath('/scratch/ucgd/lustre-labs/quinlan/data-shared/dna-methylation/CEPH1463.GRCh38.hifi.model.founder-phased')

In [4]:
from read_data import read_data

DNA_METH_FATHER = read_data(OUTPUT_DIR, f"{FATHER_UID}.dna-methylation.founder-phased")
DNA_METH_MOTHER = read_data(OUTPUT_DIR, f"{MOTHER_UID}.dna-methylation.founder-phased")
DNA_METH_CHILD = read_data(OUTPUT_DIR, f"{CHILD_UID}.dna-methylation.founder-phased")

DNA_METH_FATHER

chrom,start,end,start_hap_map_block,end_hap_map_block,haplotype_concordance_in_hap_map_block,num_het_SNVs_in_hap_map_block,methylation_level_pat,methylation_level_mat,total_read_count_pat,total_read_count_mat,founder_haplotype_pat,founder_haplotype_mat,is_within_50bp_of_mismatch_site
str,i64,i64,i64,i64,f64,i64,f64,f64,i64,i64,str,str,bool
"""chr1""",10468,10469,null,null,null,null,null,null,null,null,null,null,false
"""chr1""",10470,10471,null,null,null,null,null,null,null,null,null,null,false
"""chr1""",10483,10484,null,null,null,null,null,null,null,null,null,null,false
"""chr1""",10488,10489,null,null,null,null,null,null,null,null,null,null,false
"""chr1""",10492,10493,null,null,null,null,null,null,null,null,null,null,false
…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""chrY""",56887399,56887400,null,null,null,null,null,null,null,null,null,null,null
"""chrY""",56887579,56887580,null,null,null,null,null,null,null,null,null,null,null
"""chrY""",56887581,56887582,null,null,null,null,null,null,null,null,null,null,null


## TR DNMs in child

In [5]:
import polars as pl

pl.Config.set_tbl_rows(100)

# Tom Sasani provided TR DNMs for the child 2211: 
# https://quinlangroup.slack.com/archives/D9LFRMXV3/p1753988258836809
TR_DNMS_CHILD = pl.read_csv("/scratch/ucgd/lustre-labs/quinlan/u1006375/CEPH-K1463-TandemRepeats/tr_validation/seq/2211.GRCh38.allele_sequences.tsv", separator="\t")

(
    TR_DNMS_CHILD
    .select(['#chrom', 'start', 'end', 'phase_consensus', 'haplotype_in_parent_consensus', 'denovo_allele_sequence', 'precursor_allele_length_in_parent', 'precursor_sequence_in_parent', 'struc'])
    .with_columns([
        (pl.col('denovo_allele_sequence').str.len_chars() - pl.col('precursor_sequence_in_parent').str.len_chars()).alias('allele_size_difference')
    ])
    .rename({
        '#chrom': 'chrom',
        'phase_consensus': 'parent-of-origin', 
        'haplotype_in_parent_consensus': 'haplotype-of-origin'
    })
)

chrom,start,end,parent-of-origin,haplotype-of-origin,denovo_allele_sequence,precursor_allele_length_in_parent,precursor_sequence_in_parent,struc,allele_size_difference
str,i64,i64,str,str,str,f64,str,str,u32
"""chr10""",101793189,101793216,"""mom:64""","""UNK""","""AAAAACAAAAACAAAAACAAAAACAAAAAC…",-1.0,"""UNK""","""(AAAAAC)n""",36
"""chr10""",30432978,30433009,"""dad:279""","""B""","""AGAGAGAGAGAGAGAGAGAGAGAGAGAGAG…",247.0,"""AGAAGAGAGAGAGAGAGAGAGAGGAGAGGA…","""(GA)n""",4
"""chr10""",30433093,30433105,"""unknown:0""","""UNK""","""GAGAGAGAGAGAGAGAGAGAGAGAGAGAGA…",-1.0,"""UNK""","""(AGGGGG)n""",233
"""chr10""",44662653,44663103,"""dad:445""","""B""","""AGGGGGCTGTGTCGTGGAGGGTGAGGATGG…",6050.0,"""AGGGGGCTGTGTCGTGGAGGGTGAGGATGG…","""(GGGGTCCTAGGCGGCTGTGTCATGGAGGG…",150
"""chr10""",66515265,66515370,"""mom:287""","""B""","""TATCTATCTATCTATCTATCTATCTATCTA…",129.0,"""TATCTATCTATCTATCTATCTATCTATCTA…","""(TA)n(TC)n(TA)n""",4294967284
"""chr10""",67854303,67854574,"""dad:2""","""UNK""","""AAAAAAAAAAAAAAAAAAAAAATATATATA…",-1.0,"""UNK""","""(AT)n(AC)n(AT)n(AC)n""",274
"""chr10""",81975314,81975376,"""dad:87""","""A""","""ATCTATCTATCTATCTATCTATCTATCTAT…",66.0,"""ATCTATCTATCTATCTATCTATCTATCTAT…","""(ATCT)n""",4294967292
"""chr11""",56974116,56974221,"""dad:351""","""A""","""AACAAAAACAAAATTTGACAAGTAGGACCT…",123.0,"""AACAAAAACAAAATTTGACAAGTAGGACCT…","""(AACAAA)n(AATTA)n(AT)n""",4294967290
"""chr11""",61824394,61824531,"""mom:183""","""B""","""AAAAGAGAGAGAGAGAGAGAGAGAGAGAGA…",172.0,"""AAAGAGAGAGAGAGAGAGAGAGAGAGAGAG…","""(AG)n""",10
